<a href="https://colab.research.google.com/github/WiHi1131/Histopathologic-Cancer-Detection-Report/blob/main/Histopathic_Cancer_Detection_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd {'Histopathic-Cancer-Detection-Report'}

[Errno 2] No such file or directory: 'Histopathic-Cancer-Detection-Report'
/content


In [3]:
%ls -a

./  ../  .config/  sample_data/
